In [41]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from math import sqrt
import shap
import numpy as np

In [ ]:
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]

def random_forest_forecast(train, testX):
    # transform list into array

    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    model = RandomForestRegressor(n_estimators=1000)
    model.fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict([testX])
    return yhat[0]

def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, -1], predictions

In [94]:
day_data = pd.read_csv('../day_data.csv')
day_data['date'] = pd.to_datetime(day_data['date'], format="%Y-%m-%d")
day_data['holiday'] = day_data['holiday'].fillna('0')

onehot = OneHotEncoder(categories='auto', drop=None, sparse_output=False)
holidays = onehot.fit_transform(day_data[['holiday']])
holidays = pd.DataFrame(holidays, columns=onehot.categories_)

onehot = OneHotEncoder(categories='auto', drop='first', sparse_output=False)
wd = onehot.fit_transform(day_data[['wd']])
wd = pd.DataFrame(wd, columns=onehot.get_feature_names_out(['wd']))

day_data = pd.concat([day_data, holidays, wd], axis=1)
day_data = day_data.drop(columns=['holiday', ('0',), 'wd'])
day_data.columns = day_data.columns.astype(str)

cutoff_date = '2024-01-01'

In [95]:
train_data = day_data[day_data['date'] < cutoff_date]
train_data['x1'] = train_data['num_crimes'].shift(1)
train_data['x2'] = train_data['num_crimes'].shift(2)
train_data['x3'] = train_data['num_crimes'].shift(3)
train_data = train_data.ffill().bfill().ffill()

y_train = train_data['num_crimes']
train_dates = train_data['date']
X_train = train_data.drop(columns=['num_crimes', 'date'])

test_data = day_data[(day_data['date'] >= cutoff_date) & (day_data['date'] <= '2024-01-31')]
test_data['x1'] = test_data['num_crimes'].shift(1)
test_data['x2'] = test_data['num_crimes'].shift(2)
test_data['x3'] = test_data['num_crimes'].shift(3)
test_data = test_data.ffill().bfill().ffill()
y_test = test_data['num_crimes']
test_dates = test_data['date']
X_test = test_data.drop(columns=['num_crimes', 'date'])

C:\Users\benke\AppData\Local\Temp\ipykernel_29284\2322444573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['x1'] = train_data['num_crimes'].shift(1)
C:\Users\benke\AppData\Local\Temp\ipykernel_29284\2322444573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['x2'] = train_data['num_crimes'].shift(2)
C:\Users\benke\AppData\Local\Temp\ipykernel_29284\2322444573.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [101]:
def random_forest_forecast(train, X_test):
    y_train = train_data['num_crimes']
    train_dates = train_data['date']
    X_train = train_data.drop(columns=['num_crimes', 'date'])

    # fit model
    model = RandomForestRegressor(n_estimators=1000)
    model.fit(X_train, y_train)
    # make a one-step prediction
    yhat = model.predict(X_test)
    return yhat[0]



In [102]:
random_forest_forecast(X_train, X_test)

42.062

In [ ]:
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]

In [113]:
test_data['num_crimes'].iloc[1, :-1]

IndexingError: Too many indexers

In [114]:
def walk_forward_validation(train_data, test_data):
    predictions = []

    train = train_data
    test = test_data

    history = train

    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        y_test = test['num_crimes'].iloc[i]
        X_test = test.iloc[i]
        X_test = X_test.drop(columns=['num_crimes'])
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, X_test)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        pred_row = 
        history = pd.concat([history, test.iloc[i,:]])
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (y_test, yhat))
    # estimate prediction error
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, -1], predictions

In [98]:
rf_model = RandomForestRegressor(
    random_state=42, 
    n_estimators=1000,
    )
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse} \n RMSE: {np.sqrt(mse)}')

Mean Squared Error: 32.23210532258064 
 RMSE: 5.677332588688163


In [ ]:
walk_forward_validation